In [ ]:
"""""BACKTEST FOR EMA AND RSI TOGETEHER"""""

import pandas as pd
import numpy as np

file = ""

# A function to backtest the EMA crossover strategy
def backtest_strategy(data, emaf, emas):
    """
    Calculates the P&L for a given EMAF and EMAS combination.

    Args:
        data (pd.DataFrame): The DataFrame containing 'Close' prices.
        emaf (int): The span for the fast EMA.
        emas (int): The span for the slow EMA.

    Returns:
        float: The calculated P&L.
    """
    # Calculate the Exponential Moving Averages
    data['emaf'] = data['Close'].ewm(span=emaf, adjust=False).mean()
    data['emas'] = data['Close'].ewm(span=emas, adjust=False).mean()

    # Generate trading signals:
    # 1 for a long position (buy), -1 for a short position (sell)
    data['signal'] = 0
    data.loc[data['emaf'] > data['emas'], 'signal'] = 1
    data.loc[data['emaf'] < data['emas'], 'signal'] = -1

    # Calculate returns based on the trading signal from the previous period
    # We shift the signal by 1 to avoid look-ahead bias
    data['strategy_returns'] = data['signal'].shift(1) * data['Close'].pct_change()

    # Calculate the total P&L. The multiplier is a hypothetical position size.
    pnl = data['strategy_returns'].sum() * 1000
    return pnl

def main():
    """
    Main function to run the optimization loop.
    """
    # Load your data here. This is where you would load a CSV or other data source.
    # For this example, we're using a generated sample dataset.
    data = pd.read_csv(file)

    # Initialize variables to track the best result
    max_pnl = -np.inf  # Initialize with a very small number
    best_emaf = 0
    best_emas = 0

    # Define the range for emaf and emas
    emaf_range = range(5, 200)
    emas_range = range(5, 200)

    print("Starting backtest loop...")

    # Loop through all combinations of emaf and emas
    for emaf in emaf_range:
        for emas in emas_range:
            # Skip combinations where emaf is equal to emas
            if emaf == emas:
                continue

            # Backtest the strategy for the current combination
            current_pnl = backtest_strategy(data.copy(), emaf, emas)

            # Check if this P&L is the new maximum
            if current_pnl > max_pnl:
                max_pnl = current_pnl
                best_emaf = emaf
                best_emas = emas

    print("\nBacktest complete.")
    print("--------------------")
    # Print the final result
    print(f"The maximum P&L is: {max_pnl:.2f}")
    print(f"This was achieved with:")
    print(f"  EMAF: {best_emaf}")
    print(f"  EMAS: {best_emas}")

if __name__ == "__main__":
    main()


In [2]:
"""""BACKTEST FOR EMA CROSSOVER ONLY"""""

import pandas as pd
import numpy as np

file = r"C:\Users\Admin\My_Projects\ICICI_Direct_Analysis\DAILY DATA DUMP\SBINIF.csv"

# A function to backtest the EMA crossover strategy with RSI
def backtest_strategy(data, emaf, emas, rsi_period=None, rsi_oversold=None):
    """
    Calculates the P&L for a given EMAF, EMAS, RSI period, and RSI oversold threshold.

    Args:
        data (pd.DataFrame): The DataFrame containing 'close' prices.
        emaf (int): The span for the fast EMA.
        emas (int): The span for the slow EMA.
        rsi_period (int, optional): The lookback period for the RSI. Defaults to None.
        rsi_oversold (int, optional): The threshold for the RSI oversold condition. Defaults to None.

    Returns:
        float: The calculated P&L.
    """
    # Calculate the Exponential Moving Averages
    data['emaf'] = data['close'].ewm(span=emaf, adjust=False).mean()
    data['emas'] = data['close'].ewm(span=emas, adjust=False).mean()

    # Calculate RSI (optional)
    if rsi_period is not None and rsi_oversold is not None:
        def calculate_rsi(series, period):
            delta = series.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            avg_gain = gain.ewm(span=period, adjust=False).mean()
            avg_loss = loss.ewm(span=period, adjust=False).mean()

            # Handle division by zero if avg_loss is zero
            rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)
            rsi = 100 - (100 / (1 + rs))

            return rsi

        data['rsi'] = calculate_rsi(data['close'], rsi_period)


    # Generate trading signals:
    # 1 for a long position (buy), -1 for a short position (sell)
    data['signal'] = 0

    # Buy signal: EMA is bullish
    data.loc[(data['emaf'] > data['emas']), 'signal'] = 1

    # Sell signal: EMA is bearish
    data.loc[(data['emaf'] < data['emas']), 'signal'] = -1

    # Calculate returns based on the trading signal from the previous period
    # We shift the signal by 1 to avoid look-ahead bias
    data["signal"] = data["signal"].shift(1)
    data['strategy_returns'] = data['signal'] * data['high'].pct_change()

    # Calculate the total P&L. The multiplier is a hypothetical position size.
    pnl = data['strategy_returns'].sum() * 1000
    return pnl

def main():
    """
    Main function to run the optimization loop for EMA and RSI parameters.
    """
    # Load your data here.
    data = pd.read_csv(file)

    # Initialize variables to track the best result
    max_pnl = -np.inf  # Initialize with a very small number
    best_emaf = 0
    best_emas = 0


    # Define the range for emaf and emas parameters
    emaf_range = range(5, 200, 1) # Reduced range and increased step
    emas_range = range(5, 200, 1) # Reduced range and increased step


    print("Starting backtest loop...")

    # Loop through all combinations of parameters
    for emaf in emaf_range:
        for emas in emas_range:
            # Skip combinations where emaf is equal to or greater than emas
            if emaf >= emas:
                continue

            # Backtest the strategy for the current combination
            current_pnl = backtest_strategy(data.copy(), emaf, emas)

            # Check if this P&L is the new maximum
            if current_pnl > max_pnl:
                max_pnl = current_pnl
                best_emaf = emaf
                best_emas = emas


    print("\nBacktest complete.")
    print("--------------------")
    # Print the final result
    print(f"The maximum P&L is: {max_pnl:.2f}")
    print(f"This was achieved with:")
    print(f"  EMAF: {best_emaf}")
    print(f"  EMAS: {best_emas}")

if __name__ == "__main__":
    main()

Starting backtest loop...

Backtest complete.
--------------------
The maximum P&L is: 2597.87
This was achieved with:
  EMAF: 5
  EMAS: 23
